# Object Following - Live Demo

In this notebook we'll show how you can follow an object with JetBot!  We'll use a pre-trained neural network
that was trained on the [COCO dataset](http://cocodataset.org) to detect 90 different common objects.  These include

* Person (index 0)
* Cup (index 47)

and many others (you can check [this file](https://github.com/tensorflow/models/blob/master/research/object_detection/data/mscoco_complete_label_map.pbtxt) for a full list of class indices).  The model is sourced from the [TensorFlow object detection API](https://github.com/tensorflow/models/tree/master/research/object_detection),
which provides utilities for training object detectors for custom tasks also!  Once the model is trained, we optimize it using NVIDIA TensorRT on the Jetson Nano.

This makes the network very fast, capable of real-time execution on Jetson Nano!  We won't run through all of the training and optimization steps in this notebook though.

Anyways, let's get started.  First, we'll want to import the ``ObjectDetector`` class which takes our pre-trained SSD engine.

### Compute detections on single camera image

In [1]:
from jetbot import ObjectDetector

model = ObjectDetector('ssd_mobilenet_v2_coco.engine')

Internally, the ``ObjectDetector`` class uses the TensorRT Python API to execute the engine that we provide.  It also takes care of preprocessing the input to the neural network, as
well as parsing the detected objects.  Right now it will only work for engines created using the ``jetbot.ssd_tensorrt`` package. That package has the utilities for converting
the model from the TensorFlow object detection API to an optimized TensorRT engine.

Next, let's initialize our camera.  Our detector takes 300x300 pixel input, so we'll set this when creating the camera.

> Internally, the Camera class uses GStreamer to take advantage of Jetson Nano's Image Signal Processor (ISP).  This is super fast and offloads
> a lot of the resizing computation from the CPU. 

In [2]:
from jetbot import Camera

camera = Camera.instance(width=300, height=300)

Now, let's execute our network using some camera input.  By default the ``ObjectDetector`` class expects ``bgr8`` format that the camera produces.  However,
you could override the default pre-processing function if your input is in a different format.

In [3]:
detections = model(camera.value)

print(detections)

[[{'label': 1, 'confidence': 0.6325027346611023, 'bbox': [0.37327635288238525, 0.636114776134491, 0.5949277281761169, 0.983693540096283]}, {'label': 63, 'confidence': 0.427755206823349, 'bbox': [0.11049509048461914, 0.6950170397758484, 0.47660887241363525, 0.9947414994239807]}]]


If there are any COCO objects in the camera's field of view, they should now be stored in the ``detections`` variable.

### Display detections in text area

We'll use the code below to print out the detected objects.

In [4]:
from IPython.display import display
import ipywidgets.widgets as widgets

detections_widget = widgets.Textarea()

detections_widget.value = str(detections)

display(detections_widget)

Textarea(value="[[{'label': 1, 'confidence': 0.6325027346611023, 'bbox': [0.37327635288238525, 0.6361147761344…

You should see the label, confidence, and bounding box of each object detected in each image.  There's only one image (our camera) in this example. 


To print just the first object detected in the first image, we could call the following

> This may throw an error if no objects are detected

In [5]:
image_number = 0
object_number = 0

print(detections[image_number][object_number])

{'label': 1, 'confidence': 0.6325027346611023, 'bbox': [0.37327635288238525, 0.636114776134491, 0.5949277281761169, 0.983693540096283]}


### Control robot to follow central object

Now we want our robot to follow an object of the specified class.  To do this we'll do the following

1.  Detect objects matching the specified class
2.  Select object closest to center of camera's field of vision, this is the 'target' object
3.  Steer robot towards target object, otherwise wander
4.  If we're blocked by an obstacle, turn left

We'll also create some widgets that we'll use to control the target object label, the robot speed, and
a "turn gain", that will control how fast the robot turns based off the distance between the target object
and the center of the robot's field of view. 


First, let's load our collision detection model.  The pre-trained model is stored in this directory as a convenience, but if you followed
the collision avoidance example you may want to use that model if it's better tuned for your robot's environment.

Great, now let's initialize our robot so we can control the motors.

In [6]:
from jetbot import Robot

robot = Robot()

Finally, let's display all the control widgets and connect the network execution function to the camera updates.

In [7]:
from jetbot import bgr8_to_jpeg
import requests
import numpy as np
import cv2

blocked_widget = widgets.FloatSlider(min=0.0, max=1.0, value=0.0, description='blocked')
image_widget = widgets.Image(format='jpeg', width=300, height=300)
speed_widget = widgets.FloatSlider(value=0.1, min=0.0, max=1.0, description='speed')
turn_gain_widget = widgets.FloatSlider(value=0.2, min=0.0, max=2.0, description='turn gain')

display(widgets.VBox([
    widgets.HBox([image_widget, blocked_widget]),
    speed_widget,
    turn_gain_widget
]))

width = int(image_widget.width)
height = int(image_widget.height)

min_area = 0.4
max_area = 0.7

def detection_center(detection):
    """Computes the center x, y coordinates of the object"""
    bbox = detection['bbox']
    center_x = (bbox[0] + bbox[2]) / 2.0 - 0.5
    center_y = (bbox[1] + bbox[3]) / 2.0 - 0.5
    return (center_x, center_y)
def transfer(my_url):   #use to send and receive data
    #Replace with your sensor IP address
    ip = '192.168.1.46'
    url = 'http://' + ip + '/temp'
    response = requests.get(url)
    # Si detecta perro
    if response.text == "dog":
        label=18
    # Si detecta gato
    elif response.text == "cat":
        label=17
    # Si detecta persona
    elif response.text == "person":
        label=1
    else:
        label=0
    return label
        
def execute(change):
    image = change['new']
    mask = []
        
    # compute all detected objects
    detections = model(image)
    
    # draw all detections on image
    label = transfer("/get_color")
    if label:
        for det in detections[0]:
            if det['label'] == int(label):
                bbox = det['bbox']
                cv2.rectangle(image, (int(width * bbox[0]), int(height * bbox[1])), (int(width * bbox[2]), int(height * bbox[3])), (255, 0, 0), 2)
                # move robot forward and steer proportional target's x-distance from center
                center = detection_center(det)
                if (bbox[2]-bbox[0])*(bbox[3]-bbox[1])<min_area:
                    robot.set_motors(
                        float(speed_widget.value + turn_gain_widget.value * center[0]),
                        float(speed_widget.value - turn_gain_widget.value * center[0])
                    )
                elif (bbox[2]-bbox[0])*(bbox[3]-bbox[1])>max_area:
                    robot.set_motors(
                        float(-1*speed_widget.value - turn_gain_widget.value * center[0]),
                        float(-1*speed_widget.value + turn_gain_widget.value * center[0])
                    )
                else:
                    robot.stop()
                break
        # otherwise go forward if no target detected
        else:
            robot.stop()
    # otherwise go forward if no target detected
    else:
        robot.stop()

#     # update image widget
    image_widget.value = bgr8_to_jpeg(image)
    
execute({'new': camera.value})

Call the block below to connect the execute function to each camera frame update.

In [8]:
camera.unobserve_all()
camera.observe(execute, names='value')

Awesome!  If the robot is not blocked you should see boxes drawn around the detected objects in blue.  The target object (which the robot follows) will be displayed in green.

The robot should steer towards the target when it is detected.  If it is blocked by an object it will simply turn left.

You can call the code block below to manually disconnect the processing from the camera and stop the robot.

In [9]:
import time

camera.unobserve_all()
time.sleep(1.0)
robot.stop()